# Custom Exceptions: Tailoring Error Signals
- Built-in exceptions are great, but often too generic for application-specific failures.  
- A custom exception like `ServiceConnectionError` immediately conveys context compared to a plain `Exception`.  
- Defining a base exception class groups related errors; subclasses add specificity for targeted handling.  
- Catching `except BaseError:` handles all related issues, while `except SpecificError:` addresses one case precisely.

## Simple Custom Exceptions (Inheritance)
- Create a new exception by subclassing `Exception` or another exception class.  
- Using `pass` is enough when no extra logic or attributes are needed.  
- Catch the base class (`AutomationError`) to handle any related subclass errors in one block.  
- Use subclasses (`FileProcessingError`, `APICallError`) when context-specific handling is required.  

In [1]:
class AutomationError(Exception):
    """Base class for all automation script errors."""
    pass

class FileProcessingError(AutomationError):
    """Raised when a file processing operation fails."""
    pass 

class APICallError(AutomationError):
    """Raised when an API call fails."""
    pass 

def process_file(filepath):
    raise FileProcessingError(f"Failed to process file at path: {filepath}")


try:
    process_file("my_file.txt")
except FileProcessingError as e:
    print(f"Error processing file: {e}")
except AutomationError:
    print("Other automation error occurred")        

Error processing file: Failed to process file at path: my_file.txt


## Adding Context with `__init__`
- Override `__init__` in your exception class to capture context (e.g., filename, invalid value).  
- Store custom attributes on `self` and build a clear message passed to `super().__init__()`.  
- Inherit from a built-in exception (`ValueError`) when semantics align, allowing broad catches.  
- Attribute access (`e.key_name`) provides extra debugging info in handlers.  

In [2]:
class ConfigValueError(ValueError):
    """Raised when a configuration value is invalid."""
    def __init__(self, key_name, invalid_value, message="Invalid Config Value"):
        self.key_name = key_name
        self.invalid_value = invalid_value
        full_message = f"{message} for key '{key_name}':received '{invalid_value}'"
        super().__init__(full_message)


try:
    raise ConfigValueError("timeout", -5, message="Timeout cannot be negative")    
except ConfigValueError as e:
    print(f"Error: {e}")
    print(f"Received invalid value {e.invalid_value} for key '{e.key_name}'")

Error: Timeout cannot be negative for key 'timeout':received '-5'
Received invalid value -5 for key 'timeout'


## Raising and Catching Enhanced Custom Exceptions
- Raise custom exceptions by instantiating them with relevant arguments: `raise MyError(arg1, arg2)`.  
- In `except` blocks, catch specific exceptions and access their attributes for tailored recovery or logging.  
- Fallback `except BaseError:` catches any related subclass if no more specific handler exists.  

In [4]:
class DeploymentError(Exception):
    """Base class for deployment-related errors."""
    pass 

class InvalidEnvironmentError(DeploymentError):
    """Raised when environment is invalid."""
    def __init__(self, env_name, allowed_envs):
        self.env_name = env_name
        self.allowed_envs = allowed_envs
        super().__init__(f"Invalid environment '{env_name}'. Allowed environments: {allowed_envs}")

class PackageMissingError(DeploymentError):
    """Raised when a required package is missing."""
    def __init__(self, package_name, host):
        self.package_name = package_name
        self.host = host
        super().__init__(f"Missing package '{package_name}' on host '{host}'.")   


def deploy_application(environment, package):         
    allowed_envs = ["dev", "test", "prod"]
    if environment not in allowed_envs:
        raise InvalidEnvironmentError(environment, allowed_envs)
    
    if environment == "prod" and package == "critical-lib":
        raise PackageMissingError(package, f"server-{environment}")
    
    print(f"Deployment to environment '{environment}' with package '{package}' succeeded.")
    

for env, pkg in [("dev", "my-app"), ("test", "tool"), ("prod", "critical-lib"), ("prod", "tool")]:  
    try:
        deploy_application(env, pkg)
    except InvalidEnvironmentError as e:
        print(f"Error: {e}")
    except PackageMissingError as e:
        print(f"Error: {e}")
    except DeploymentError as e:
        print(f"Error: {e}")
    

Deployment to environment 'dev' with package 'my-app' succeeded.
Deployment to environment 'test' with package 'tool' succeeded.
Error: Missing package 'critical-lib' on host 'server-prod'.
Deployment to environment 'prod' with package 'tool' succeeded.
